In [1]:
import numpy as np
import pandas as pd
import os
import requests
from tqdm.notebook import tqdm

In [2]:
def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
 
    data_type can be 'comment' or 'submission'
    The rest of the args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    """
 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.json()

In [18]:
os.mkdir("data/submissions/robinhoodpennystocks")

In [19]:
dates = pd.date_range("20190815", "20200815", tz="US/Eastern")

subreddits = [
    "wallstreetbets",
    "robinhood",
    "pennystocks",
    "robinhoodpennystocks",
]

for start, end in tqdm(zip(dates[:-1], dates[1:])):
    for sub in tqdm(subreddits, leave=False):
        submissions_fname = f"data/submissions/{sub}/{start.strftime('%Y%m%d')}.pickle"
        comments_fname = f"data/comments/{sub}/{start.strftime('%Y%m%d')}.pickle"
        
        if os.path.exists(submissions_fname) and os.path.exists(comments_fname):
            continue
        try:
            submissions = get_pushshift_data(
                data_type="submission",
                after=int(start.value / 10 ** 9),
                before=int(end.value / 10 ** 9),
                size=25,
                sort_type="score",
                sort="desc",
                subreddit=sub,
            )
        except:
            print(f"Failed to retrieve submissions for {sub} on {start}")
            continue
        submissions = pd.DataFrame(submissions["data"])
        submissions.to_pickle(submissions_fname)
        
        all_comments = []
        failed = False
        for submission_id in tqdm(submissions.id, leave=False):
            try:
                comments = get_pushshift_data(
                    data_type="comment",
                    size=25,
                    sort_type="score",
                    sort="desc",
                    link_id=submission_id,
                )
            except:
                print(f"Failed to retrieve comments for {sub} post {submission_id} on {start}")
                failed=True
                break
            comments = pd.DataFrame(comments["data"])
            all_comments.append(comments)
        if failed:
            continue
        all_comments = pd.concat(all_comments)
        all_comments.to_pickle(comments_fname)

AttributeError: 'DataFrame' object has no attribute 'id'